In [ ]:
!pip install ultralytics wandb

In [ ]:
!wandb login

In [ ]:
from ultralytics import YOLO
from ray import tune

### Обучение

In [ ]:
model = YOLO('yolov8n.pt')

In [ ]:
model.train(
        data='signs4/data.yaml',
        imgsz=640,
        batch=16,
        epochs=200,
        optimizer= 'Adam',
        seed=0,
        pretrained=True,
        name='yolov8'
        )

### Оценка качества

In [ ]:
m = YOLO('runs/detect/train2/weights/best.pt')

metrics = m.val(data = 'signs4/data.yaml', split='test')

### Подбор гиперпараметров

In [ ]:
result_grid = model.tune(data='/signs4/data.yaml',
                         use_ray=True,
                         epochs=100,
                         imgsz=640,
                         batch=16,
                         optimizer='Adam',
                         seed=0,
                         pretrained=True,
                         space={"lr0": tune.uniform(1e-5, 1e-1),
                               "lrf": tune.uniform(0.01, 1.0),
                               "momentum": tune.uniform(0.6, 0.98),
                               "weight_decay": tune.uniform(0.0, 0.001),
                               "warmup_epochs": tune.uniform(0.0, 5.0),
                               "warmup_momentum": tune.uniform(0.0, 0.95),
                               "box": tune.uniform(0.02, 0.2),
                               "cls": tune.uniform(0.2, 4.0),
                               },
                         )

### Экспорт модели в формат onnx

In [ ]:
model.export(format='onnx', dynamic=True, simplify=True)